In [3]:
from mp_api.client import MPRester
from pymatgen.analysis.pourbaix_diagram import PourbaixDiagram, PourbaixPlotter, PourbaixEntry


API_KEY = "M3iWLItVErrMW5qgPwe5tzZFxjlROt1x"  # Replace with your actual API key

In [4]:
materials = ["mp-2657", "mp-4812"]
mp_entries = []

with MPRester(API_KEY) as mpr:
    for mp_id in materials:
        # Search for materials and expect a list of results
        material_entries = mpr.materials.search(material_ids=[mp_id])

        # Check if the search returned any results
        if material_entries:
            # Append only the first MPDataDoc object from the results
            mp_entries.append(material_entries[0])

# Now extract the 'chemsys' attribute from each entry in mp_entries
chemsys_list = [entry.chemsys for entry in mp_entries if hasattr(entry, 'chemsys')]
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')]
materials_gga = [material + "-GGA" for material in materials]

# Print the chemsys attribute of each entry
print(chemsys_list)
print(composition_list)
print(materials_gga)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['O-Ti', 'B-O-Zn']
[Composition('Ti1 O2'), Composition('Zn4 B6 O13')]
['mp-2657-GGA', 'mp-4812-GGA']


In [8]:
pbx_data = []
pbx_entries = []

with MPRester(API_KEY) as mpr:
    for i in chemsys_list:
        pourbaix_entries = mpr.get_pourbaix_entries(i)
        pbx_entries.append(pourbaix_entries)
        pbx_diagram = PourbaixDiagram(pourbaix_entries)
        pbx_data.append(pbx_diagram)  

Retrieving ThermoDoc documents:   0%|          | 0/218 [00:00<?, ?it/s]

Retrieving ThermoDoc documents:   0%|          | 0/184 [00:00<?, ?it/s]

In [6]:
matched_entries = []

for mat_id, entries in zip(materials_gga, pbx_entries):
    matched_entry = next((entry for entry in entries if entry.entry_id == mat_id), None)
    
    # Add the matched entry to the new list
    if matched_entry:
        matched_entries.append(matched_entry)
    else:
        # If no match is found, append None or handle as required
        matched_entries.append(None)

# 'matched_entries' now contains the matched PourbaixEntry objects or None for no match
print(matched_entries)

[PourbaixEntry(Ti2 O4 with energy = -9.6864, npH = -8.0, nPhi = -8.0, nH2O = 4.0, entry_id = 'mp-2657-GGA'), PourbaixEntry(Zn4 B6 O13 with energy = -21.6911, npH = -26.0, nPhi = -26.0, nH2O = 13.0, entry_id = 'mp-4812-GGA')]


---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[7], line 5
      3 with MPRester(API_KEY) as mpr:
      4     for i in matched_entries:
----> 5         pbx_diagram = PourbaixDiagram(i)
      6         pbx_diagrams.append(pbx_diagram)
      7 print(pbx_diagrams[0])

File ~/anaconda3/lib/python3.11/site-packages/pymatgen/analysis/pourbaix_diagram.py:448, in PourbaixDiagram.__init__(self, entries, comp_dict, conc_dict, filter_solids, nproc)
    444 self.filter_solids = filter_solids
    446 # Get non-OH elements
    447 self.pbx_elts = list(
--> 448     set(itertools.chain.from_iterable([entry.composition.elements for entry in entries])) - ELEMENTS_HO
    449 )
    450 self.dim = len(self.pbx_elts) - 1
    452 # Process multi-entry inputs

TypeError: 'PourbaixEntry' object is not iterable


In [ ]:
#pbx_diagram = PourbaixDiagram(pourbaix_entries)

In [ ]:
#ti_o2_entry = None
#for entry in pourbaix_entries:
   # if entry.entry_id == "mp-390-GGA":  # Match the entry_id
       # ti_o2_entry = entry
     #   break#

In [ ]:
#plotter = PourbaixPlotter(pbx_diagram)
#plotter.plot_entry_stability(ti_o2_entry)

In [ ]:
#pbx_entry = PourbaixEntry(ti_o2_entry)
#stability_data = pbx_diagram.get_decomposition_energy(ti_o2_entry, 7, 0)
#print(stability_data)

0.006397966666667069
